### Modélisation 

L’objectif de cette section est d’évaluer l’impact du rééquilibrage par **SMOTE** (appliqué uniquement sur le jeu d’entraînement) sur les performances de modèles de classification.  

Nous comparons deux modèles : **régression logistique** et **Random Forest**, appliquée sans rééquilibrage de la base de données et avec réequilibrage.

Dans un contexte de classes déséquilibrées, le ROC-AUC ne suffit pas toujours : nous analysons également les matrices de confusion afin d’étudier le compromis entre :
- FN : résiliations non détectées (coût potentiellement élevé),
- FP : fausses alertes (coût commercial / opérationnel).


### 1- Importation des données et packages 

In [57]:
from pathlib import Path
import pandas as pd
import numpy as np
import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report, roc_auc_score, confusion_matrix,
    precision_score, recall_score, f1_score, average_precision_score
)

from imblearn.over_sampling import SMOTE

In [58]:
IN_DIR = Path("..") / "data" / "processed"

data = np.load(IN_DIR / "eudirectlapse_num_scaled_split.npz", allow_pickle=True)

X_train_num_scaled = data["X_train"]
X_test_num_scaled = data["X_test"]
y_train = data["y_train"]
y_test = data["y_test"]
num_cols = data["num_cols"].tolist()

scaler = joblib.load(IN_DIR / "scaler_num.joblib")

print(X_train_num_scaled.shape, X_test_num_scaled.shape)

(17295, 9) (5765, 9)


In [59]:
print("Train target rate:", y_train.mean())
print("Test target rate:", y_test.mean())

Train target rate: 0.1281295172015033
Test target rate: 0.1280138768430182


### 1- Régression logistique 
La régression logistique est un modèle linéaire simple et interprétable.  
Dans un contexte de déséquilibre, elle peut privilégier la classe majoritaire, ce qui se traduit par un faible nombre de résiliations détectées.

#### a - Modèle de référence : Régression logistique (sans SMOTE)

In [60]:
model1 = LogisticRegression(max_iter=1000)
model1.fit(X_train_num_scaled, y_train)

LogisticRegression(max_iter=1000)

Le paramètre max_iter est fixé à 1000 afin de garantir la convergence de l’algorithme d’optimisation, en particulier dans un contexte de données déséquilibrées et standardisées.

In [61]:
y_pred_base = model1.predict(X_test_num_scaled)
y_proba_base = model1.predict_proba(X_test_num_scaled)[:, 1]

La fonction predict() impose que le seuil de décision est fixé à 0.5. Dans un contexte de classes déséquilibrées, ce seuil peut être inadapté, et son ajustement permet de contrôler le compromis entre faux négatifs et faux positifs.

Le seuil 0.5 n’est pas forcément optimal en cas de classes déséquilibrées. Ajuster le seuil permet de contrôler le compromis FN/FP.

In [62]:
roc_auc = roc_auc_score(y_test, y_proba_base)

# Tableau récapitulatif
metrics_summary = pd.DataFrame({
    "Metric": ["ROC-AUC"],
    "Value": [roc_auc]
})

print("=== Performance summary ===")
display(metrics_summary)

print("=== Confusion matrix ===")
display(pd.DataFrame(
    confusion_matrix(y_test, y_pred_base),
    index=["True 0", "True 1"],
    columns=["Pred 0", "Pred 1"]
))

print("=== Classification report ===")
print(classification_report(y_test, y_pred_base, digits=4))


=== Performance summary ===


,Metric,Value
0,ROC-AUC,0.585924


=== Confusion matrix ===


,Pred 0,Pred 1
True 0,5026,1
True 1,738,0


=== Classification report ===
              precision    recall  f1-score   support

           0     0.8720    0.9998    0.9315      5027
           1     0.0000    0.0000    0.0000       738

    accuracy                         0.8718      5765
   macro avg     0.4360    0.4999    0.4658      5765
weighted avg     0.7603    0.8718    0.8123      5765



Avec un ROC-AUC de 0,586, la performance globale du modèle sans rééquilibrage reste modérée. L’analyse des métriques comme la précision, le rappel et le F1-score, ainsi que l’analyse de la matrice de confusion mettent en évidence l'incapacité du modèle à détecter des résiliations.

En effet, le modèle prédit quasi exclusivement la classe majoritaire : aucune résiliation n’est correctement identifiée (TP = 0), tandis qu’une seule observation est prédite à tort comme résiliation. La précision et le rappel associés à la classe 1 sont ainsi nuls, indiquant une incapacité du modèle à reconnaître les contrats susceptibles de résilier.

Dans ce contexte de classes déséquilibrées, l’accuracy élevée (87,2 %) est trompeuse, car elle reflète essentiellement la bonne prédiction de la classe majoritaire. Ces résultats soulignent l’importance d’étudier d’autres métriques en présence de déséquilibre dans la distribution de la variable à expliquer.

#### b - Régression logistique avec rééquilibrage (SMOTE)

Dans cette section, nous évaluons l’impact du rééquilibrage des classes par la méthode SMOTE sur les performances d’une régression logistique.  
SMOTE est appliqué uniquement sur le jeu d’entraînement afin d’éviter toute fuite d’information vers le jeu de test.

In [63]:
smote = SMOTE(random_state=42)

X_train_smote, y_train_smote = smote.fit_resample(
    X_train_num_scaled,
    y_train
)

print("Distribution après SMOTE :")
print(pd.Series(y_train_smote).value_counts(normalize=True))

Distribution après SMOTE :
0    0.5
1    0.5
Name: proportion, dtype: float64


Après application de SMOTE, la distribution de la variable *lapse* dans le jeu
d’entraînement est équilibrée. Les observations synthétiques sont générées
uniquement à partir des données d’apprentissage. On peut à nouveau effectuer une régression logistique sur cette nouvelles base des données. 


In [64]:
model_smote = LogisticRegression(max_iter=1000)
model_smote.fit(X_train_smote, y_train_smote)

LogisticRegression(max_iter=1000)

In [65]:
y_pred_smote = model_smote.predict(X_test_num_scaled)
y_proba_smote = model_smote.predict_proba(X_test_num_scaled)[:, 1]

In [66]:
roc_auc = roc_auc_score(y_test, y_proba_smote)

metrics_smote = pd.DataFrame({
    "Metric": ["ROC-AUC"],
    "Value": [roc_auc]
})

print("=== Logistic Regression + SMOTE — Performance summary ===")
display(metrics_smote)

print("=== Confusion matrix ===")
display(pd.DataFrame(
    confusion_matrix(y_test, y_pred_smote),
    index=["True 0", "True 1"],
    columns=["Pred 0", "Pred 1"]
))

print("=== Classification report ===")
print(classification_report(y_test, y_pred_smote, digits=4))

=== Logistic Regression + SMOTE — Performance summary ===


,Metric,Value
0,ROC-AUC,0.586548


=== Confusion matrix ===


,Pred 0,Pred 1
True 0,2735,2292
True 1,316,422


=== Classification report ===
              precision    recall  f1-score   support

           0     0.8964    0.5441    0.6771      5027
           1     0.1555    0.5718    0.2445       738

    accuracy                         0.5476      5765
   macro avg     0.5260    0.5579    0.4608      5765
weighted avg     0.8016    0.5476    0.6218      5765



L’application de SMOTE modifie le comportement de la régression logistique. Le ROC-AUC reste du même ordre de grandeur que pour le modèle sans SMOTE (0,587 contre 0,586). L’analyse des métriques usuels sur la classe minoritaire met en évidence une amélioration de la détection des résiliations.

La matrice de confusion montre que le modèle après rééquilibrage identifie correctement 422 résiliations (TP), contre aucune pour le modèle sans SMOTE. Le rappel de la classe minoritaire atteint ainsi 57,2 %, traduisant une réduction importante du nombre de faux négatifs. On remarque aussi que cette amélioration s’accompagne d’une augmentation du nombre de faux positifs (2 292).

Dans un contexte de classes déséquilibrées, la diminution de l’accuracy globale (54,8 %) n’est pas problématique, car elle reflète le changement d’équilibre entre les classes. Ces résultats confirment que SMOTE permet de rendre le modèle sensible à la classe d’intérêt, au prix d’un compromis entre détection des résiliations et augmentation des fausses alertes.

### 3 - Random Forest 
Dans cette section, nous évaluons les performances d’un modèle de Random Forest, d’abord sans rééquilibrage des classes, puis avec application de SMOTE sur le jeu d’entraînement.

La Random Forest est un modèle non-linéaire basé sur des arbres de décison. Elle est souvent plus flexible, mais peut également être affectée par le déséquilibre des classes.  

#### a- Modèle de référence : Random Forest sans SMOTE



In [67]:
rf_base = RandomForestClassifier(
    n_estimators=500,
    random_state=42,
    n_jobs=-1
)

rf_base.fit(X_train_num_scaled, y_train)

RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=42)

In [68]:
y_pred_rf_base = rf_base.predict(X_test_num_scaled)
y_proba_rf_base = rf_base.predict_proba(X_test_num_scaled)[:, 1]

In [74]:
roc_auc = roc_auc_score(y_test, y_proba_rf_base)

metrics_rf_base = pd.DataFrame({
    "Metric": ["ROC-AUC"],
    "Value": [roc_auc]
})

print("=== Random Forest — Performance summary ===")
display(metrics_rf_base)

print("=== Confusion matrix ===")
display(pd.DataFrame(
    confusion_matrix(y_test, y_pred_rf_base),
    index=["True 0", "True 1"],
    columns=["Pred 0", "Pred 1"]
))

print("=== Classification report ===")
print(classification_report(y_test, y_pred_rf_base, digits=4))


=== Random Forest — Performance summary ===


,Metric,Value
0,ROC-AUC,0.548891


=== Confusion matrix ===


,Pred 0,Pred 1
True 0,5015,12
True 1,736,2


=== Classification report ===
              precision    recall  f1-score   support

           0     0.8720    0.9976    0.9306      5027
           1     0.1429    0.0027    0.0053       738

    accuracy                         0.8703      5765
   macro avg     0.5074    0.5002    0.4680      5765
weighted avg     0.7787    0.8703    0.8122      5765



Comme la régression logistique, l’algorithme de random forest entraîné sans rééquilibrage a tendance à privilégier la classe majoritaire et peine à détecter les résiliations.

#### b- Random Forest avec SMOTE

On applique SMOTE aux données d’entraînement (variables numériques standardisées), puis on entraîne la Random Forest sur ce jeu de données rééquilibré. 

In [ ]:
smote = SMOTE(random_state=42)
X_train_rf_smote, y_train_rf_smote = smote.fit_resample(X_train_num_scaled, y_train)

rf_smote = RandomForestClassifier(
    n_estimators=500,
    random_state=42,
    n_jobs=-1
)

rf_smote.fit(X_train_rf_smote, y_train_rf_smote)

y_pred_rf_smote = rf_smote.predict(X_test_num_scaled)
y_proba_rf_smote = rf_smote.predict_proba(X_test_num_scaled)[:, 1]

In [76]:
roc_auc = roc_auc_score(y_test, y_proba_rf_smote)

metrics_rf_smote = pd.DataFrame({
    "Metric": ["ROC-AUC"],
    "Value": [roc_auc]
})

print("=== Random Forest + SMOTE — Performance summary ===")
display(metrics_rf_smote)

print("=== Confusion matrix ===")
display(pd.DataFrame(
    confusion_matrix(y_test, y_pred_rf_smote),
    index=["True 0", "True 1"],
    columns=["Pred 0", "Pred 1"]
))

print("=== Classification report ===")
print(classification_report(y_test, y_pred_rf_smote, digits=4))

=== Random Forest + SMOTE — Performance summary ===


,Metric,Value
0,ROC-AUC,0.545322


=== Confusion matrix ===


,Pred 0,Pred 1
True 0,4895,132
True 1,705,33


=== Classification report ===
              precision    recall  f1-score   support

           0     0.8741    0.9737    0.9212      5027
           1     0.2000    0.0447    0.0731       738

    accuracy                         0.8548      5765
   macro avg     0.5371    0.5092    0.4972      5765
weighted avg     0.7878    0.8548    0.8127      5765



L’application de SMOTE à la base d'apprentissage améliore la capacité de la Random Forest à détecter la classe minoritaire, comme en témoigne l’augmentation du nombre de résiliations correctement identifiées (TP). Toutefois, le gain reste plus limité que pour la régression logistique, et s’accompagne d’une augmentation du nombre de faux positifs.

Ces résultats suggèrent que, dans ce jeu de données, la Random Forest bénéficie moins du rééquilibrage par SMOTE que la régression logistique.


L’ensemble des résultats et des analyses présentés dans ce notebook sera synthétisé et discuté de manière approfondie dans le rapport final, qui proposera une mise en perspective des choix méthodologiques, des limites observées et des pistes d’amélioration possibles.